In [62]:
import requests
import pandas as pd
import urllib.parse
import numpy as np
import re

In [63]:
make = 'audi'
model = 'a3'
yearMin = 2010
yearMax = 2020
kmMin = 0
kmMax = 100000
priceMin = 0
priceMax = 10000

'https://api.walla.com/api/v3/cars/search?filters_source=suggester&keywords=audi%20a4&category_ids=100&longitude=-3.69196&latitude=40.41956'
url_base = 'https://api.wallapop.com/api/v3/cars/search?'

parametros = {
    'filters_source': 'suggester',
    'keywords': make + ' ' + model,
    'category_ids': 100,
    'longitude': -3.69196,
    'latitude': 40.41956,
    'yearMin': yearMin,
    'yearMax': yearMax,
    'kmMin': kmMin,
    'kmMax': kmMax,
    'priceMin': priceMin,
    'priceMax': priceMax
}
url_completa = url_base + urllib.parse.urlencode(parametros)
response = requests.get(url_completa)
if response.ok:
    try:
        data = response.json()
        rows = []
        for obj in data['search_objects']:
            row = obj['content']
            rows.append(row)

        df = pd.DataFrame(rows)
    except Exception as e:
        print("Error:", e)


In [64]:
df = df[['title', 'price', 'km', 'engine', 'gearbox', 'horsepower']]

def scrape_year(title):
    # Using regular expression to get a 4-digit combination starting with 1 or 2
    pattern = r'\b(?:1|2)\d{3}\b'
    match = re.search(pattern, title)
    if match:
        year = match.group(0)
        return int(year)
    else:
        return None


df['year'] = df['title'].apply(scrape_year)
df = df.dropna()
df['environmental_label'] = np.nan
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 3 to 39
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                16 non-null     object 
 1   price                16 non-null     float64
 2   km                   16 non-null     float64
 3   engine               16 non-null     object 
 4   gearbox              16 non-null     object 
 5   horsepower           16 non-null     float64
 6   year                 16 non-null     float64
 7   environmental_label  0 non-null      float64
dtypes: float64(5), object(3)
memory usage: 1.1+ KB


In [2]:
import urllib.parse
import requests
import pandas as pd
import numpy as np

In [4]:
make = 'audi'
model = 'a3'
yearMin = 2010
yearMax = 2020
kmMin = 0
kmMax = 100000
priceMin = 0
priceMax = 100000

parametros = {
    'filters_source': 'suggester',
    'keywords': make + ' ' + model,
    'category_ids': 100,
    'longitude': -3.69196,
    'latitude': 40.41956,
    'yearMin': yearMin,
    'yearMax': yearMax,
    'kmMin': kmMin,
    'kmMax': kmMax,
    'priceMin': priceMin,
    'priceMax': priceMax
}

url_base = 'https://api.wallapop.com/api/v3/cars/search?'
url_completa = url_base + urllib.parse.urlencode(parametros)
response = requests.get(url_completa)
print(response)
df_walla = pd.DataFrame()
if response.ok:
    try:
        data = response.json()
        rows = []
        for obj in data['search_objects']:
            row = obj['content']
            rows.append(row)
        df_walla = pd.DataFrame(rows)
    except Exception as e:
        print("Error:", e)
print(df_walla.columns)

<Response [200]>
Index(['id', 'title', 'storytelling', 'distance', 'images', 'user', 'flags',
       'visibility_flags', 'price', 'currency', 'web_slug', 'category_id',
       'brand', 'model', 'year', 'version', 'km', 'engine', 'gearbox',
       'favorited', 'creation_date', 'modification_date', 'location',
       'shipping', 'supports_shipping', 'horsepower'],
      dtype='object')


In [5]:
df_walla.head(20)

,id,title,storytelling,distance,images,user,flags,visibility_flags,price,currency,...,km,engine,gearbox,favorited,creation_date,modification_date,location,shipping,supports_shipping,horsepower
0,3zl88w3m546x,Audi A3 2.0 FSI,Audi 3 - 2.0 FSI\nCoche de UN SOLO DUEÑO y sie...,10.0,[{'original': 'https://cdn.wallapop.com/images...,"{'micro_name': '', 'id': 'kmzno7xl5gjn', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': True, 'highlighted': False, 'urgent...",5500.0,EUR,...,218000.0,Gasoline,manual,False,2024-04-15T16:53:32.969+00:00,2024-05-09T06:56:42.274+00:00,"{'postal_code': '28223', 'country_code': 'ES',...","{'item_is_shippable': False, 'user_allows_ship...",False,NaN
1,ejk8m84xwpjx,Audi A3 Sportback 30 TFSI 85 kW (116 CV),Precio oferta sujeto a financiación: 20445 EUR...,15.0,[{'original': 'https://cdn.wallapop.com/images...,"{'micro_name': '', 'id': '8j3y83q89169', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': True, 'highlighted': False, 'urgent...",20445.0,EUR,...,38996.0,Gasoline,manual,False,2024-04-29T00:06:53.467+00:00,2024-05-09T08:00:40.135+00:00,"{'postal_code': '28906', 'country_code': 'ES',...","{'item_is_shippable': False, 'user_allows_ship...",False,116.0
2,wzv4ggm821zl,Audi A3 Sportback Advanced 35 TFSI 110 kW (150...,Precio oferta sujeto a financiación: 24272 EUR...,15.0,[{'original': 'https://cdn.wallapop.com/images...,"{'micro_name': '', 'id': '8j3y83q89169', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': True, 'highlighted': False, 'urgent...",24272.0,EUR,...,35681.0,Gasoline,manual,False,2024-05-01T01:51:37.184+00:00,2024-05-09T08:00:37.482+00:00,"{'postal_code': '28906', 'country_code': 'ES',...","{'item_is_shippable': False, 'user_allows_ship...",False,150.0
3,nz048rg10djo,Audi A3 2019 S LINE S-TONIC NACIONAL,*****VEHICULO NACIONAL ****IVA DEDUCIBLE EN FA...,2.0,[{'original': 'https://cdn.wallapop.com/images...,"{'micro_name': '', 'id': 'pj9k3lepnoze', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': True, 'highlighted': False, 'urgent...",19500.0,EUR,...,192000.0,Gasoil,automatic,False,2024-05-01T09:21:10.255+00:00,2024-05-08T22:00:11.729+00:00,"{'postal_code': '28070', 'country_code': 'ES',...","{'item_is_shippable': False, 'user_allows_ship...",False,150.0
4,qzmemn1pggjv,Audi A3 2011,¡Increíble oportunidad! Vendo Audi A3 S.line 2...,5.0,[{'original': 'https://cdn.wallapop.com/images...,"{'micro_name': '', 'id': 'nzxlvk9n95z2', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': True, 'highlighted': False, 'urgent...",8999.0,EUR,...,198000.0,Gasoil,automatic,False,2024-05-05T11:39:48.128+00:00,2024-05-05T11:46:12.878+00:00,"{'postal_code': '28039', 'country_code': 'ES',...","{'item_is_shippable': False, 'user_allows_ship...",False,140.0
5,xzo2krken569,Audi A3 Sportback Advanced 40 TFSI e 150kW S t...,Audi A3 Sportback Advanced 40 TFSI e 150kW S t...,15.0,[{'original': 'https://cdn.wallapop.com/images...,"{'micro_name': '', 'id': 'qnzxqg843m62', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': True, 'highlighted': False, 'urgent...",32990.0,EUR,...,47500.0,NaN,automatic,False,2023-11-18T01:00:19.865+00:00,2024-05-08T18:17:02.385+00:00,"{'postal_code': '28520', 'country_code': 'ES',...","{'item_is_shippable': False, 'user_allows_ship...",False,204.0
6,qjw4y7emykzo,AUDI A3 S LINE EDITION 1.6 TDI SPORTBACK,Pide un VIDEO por WhatsApp. Garantía PREMIUM. ...,0.0,[{'original': 'https://cdn.wallapop.com/images...,"{'micro_name': '', 'id': '8j3yewd4k169', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': True, 'highlighted': False, 'urgent...",18990.0,EUR,...,115000.0,Gasoil,manual,False,2024-01-17T03:02:30.488+00:00,2024-05-08T10:30:55.368+00:00,"{'postal_code': '28014', 'country_code': 'ES',...","{'item_is_shippable': False, 'user_allows_ship...",False,110.0
7,3zl88qvwl46x,Audi A3 SPORT BACK SPORT BACK 1.6 TDI c 2013,Único Dueño - Impe

In [67]:
parametros = {
        'tipo_busqueda': '2',
        'seminuevo': '0',
        'ord[]': 'marca_up',
        'searched3': '',
        'color': '',
        'combustible_id': '',
        'precio_desde': priceMin,
        'precio_hasta': priceMax,
        'scf_fee_desde': '',
        'scf_fee_hasta': '',
        'potencia_desde': '',
        'potencia_hasta': '',
        'km_desde': kmMin,
        'km_hasta': kmMax,
        'anyo_desde': yearMin,
        'anyo_hasta': yearMax,
        'cambio': '',
        'puertas': '',
        'plazas': '',
        'vendedor': '',
        'make_list[]': '',
        'transmission_name': '',
        'agent_type_name': '',
        'has_financing': '',
        'reservable': '',
        'search3': make + ' ' + model,
    }
url_base = 'https://www.coches.com/api/vo/pills/?'
url_completa = url_base + urllib.parse.urlencode(parametros)
response = requests.get(url_completa)
if response.ok:
    try:
        data = response.json()
        df_com = pd.DataFrame(data['pills'])
    except:
        print("Error:", e)

Index(['title', 'price', 'km', 'engine', 'gearbox', 'horsepower', 'year',
       'environmental_label'],
      dtype='object')
16


In [69]:
same_columns_wallapop = ['id', 'title', 'images', 'price', 'brand', 'model', 'year', 'km', 'engine', 'horsepower']
same_columns_coches_com = ['id','image', 'price', 'url' , 'make', 'model' , 'fuel', 'cv', 'km', 'year']
comon_names = ['id','image','price','make','model','year','km','fuelType','horsepower','cv','url']
rename_columns = {
    'engine': 'fuelType',
    'fuel' : 'fuelType',
    'horsepower': 'cv',
    'brand' : 'make',
    'images': 'image',
    'url': 'link'
}
delete_columns = ['title']
df_walla['images'] = df_walla['images'].apply(lambda x: x[0]['original'] if isinstance(x, list) and len(x) > 0 else None)
df_walla.rename(columns=rename_columns, inplace=True)

df_walla.drop(columns=delete_columns, inplace=True, errors='ignore')

df_com.rename(columns=rename_columns, inplace=True)

df_com.drop(columns=delete_columns, inplace=True, errors='ignore')

df_walla = df_walla[[col for col in comon_names if col in df_walla.columns]]
df_com = df_com[[col for col in comon_names if col in df_com.columns]]

df_walla['site'] = 'walla'

df_com['site'] = 'com'


In [72]:
print(len(df_walla))
df_walla.head(1)

40


,id,image,price,make,model,year,km,fuelType,cv,site
0,3zl88w3m546x,https://cdn.wallapop.com/images/10420/gj/wi/__...,5500.0,Audi,A3 2.0 FSI,2004,218000.0,Gasoline,NaN,walla


In [73]:
print(len(df_com))
df_com.head(1)

20


,id,image,price,make,model,year,km,fuelType,cv,site
0,8125894,https://images.coches.com/_vo_/fotos/usados/20...,26.900 €,Audi,A3,2014,75.000,Gasolina,300,com
